In [8]:
import requests
import re
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup as bsoup
from collections import defaultdict

In [1]:
import urllib
from selenium import webdriver
from selenium.webdriver.common.keys import Keys

In [ ]:
fp = webdriver.FirefoxProfile()
fp.set_preference("browser.startup.homepage_override.mstone", "ignore") #Avoid startup screen
fp.set_preference("startup.homepage_welcome_url.additional",  "about:blank")

driver = webdriver.Firefox(firefox_profile=fp, executable_path=r"C:\Users\Makarand\Anaconda3\Lib\site-packages\selenium\webdriver\firefox\geckodriver.exe")
driver.set_window_size(1120, 550)
driver.set_page_load_timeout(30)

In [40]:
## Test cell
queries = ["packers","bears","eagles","cowboys","raiders","chiefs"]
nfl_info = defaultdict(list)

for query in queries:
    page_url = "https://www.si.com/nfl/" + query
    nfl_info = NFLscrape(nfl_info, query)
    
test_df = pd.DataFrame(nfl_info)

In [95]:
def SI_NFLscrape(nfl_info, query, driver):
    page_url = "https://www.si.com/nfl/" + query

    response = requests.get(page_url)
    soup = bsoup(response.text, 'lxml')
    si_url = re.compile("/nfl/"+ query+"/\w+")

    for posting in set(soup.find_all('a')):
        article_contents = ""
        article_url = posting.get('href')
        if si_url.match(str(article_url)):
            article_url = "https://www.si.com" + article_url
            #article_response = requests.get(article_url)
            #article_soup = bsoup(response.text, 'lxml')
            #article_title = article_soup.find("div", {"class": "story"}).h1.text
            #article_author = article_soup.find_all('p', {"class":"skpzhulc sk3x58id skh9fqbk"})

            try:
                driver.get(article_url)
                article_title = driver.find_element_by_css_selector('h1').text
            except: 
                continue
            texts = driver.find_elements_by_css_selector('p')
            for text in texts:
                try:
                    if text.text != "Read More":
                        article_contents += text.text
                except:
                    continue  

            #for text in texts:
            #    if text.text != "Read More": #and text.text not in [author.text for author in article_author]:
            #        article_contents += text.text        
            nfl_info["Team"].append(query)
            nfl_info["Article URL"].append(article_url)
            nfl_info["Article Title"].append(article_title)
            #nfl_info["Article Author"].append(article_author[0])
            nfl_info["Text"].append(article_contents)           
    
    return nfl_info

In [97]:
def ESPN_NFLscrape(nfl_info, query, team_dict):
    page_url = "https://www.espn.com/blog/" + query

    response = requests.get(page_url)
    soup = bsoup(response.text, 'lxml')

    article_contents = ""
    article_url = page_url
    article_title = soup.find("header", {"class": "article-header"}).text
    #article_author = article_soup.find_all('div', {"class":"author has-bio"})
    texts = soup.find_all('p')
    
    for text in texts:
        article_contents += text.text        
    nfl_info["Team"].append(team_dict[query])
    nfl_info["Article URL"].append(article_url)
    nfl_info["Article Title"].append(article_title)
    #nfl_info["Article Author"].append(article_author[0])
    nfl_info["Text"].append(article_contents)           

    return nfl_info

In [98]:
def SBNation_NFLscrape(nfl_info, query, team_dict, driver):
    page_url = query

    response = requests.get(page_url)
    soup = bsoup(response.text, 'lxml')
    sb_url = re.compile(query+"\w+")

    for posting in set(soup.find_all('a')):
        article_contents = ""
        article_url = posting.get('href')
        if sb_url.match(str(article_url)):
            print(article_url)
            #article_response = requests.get(article_url)
            #article_soup = bsoup(response.text, 'lxml')
            #print(article_soup.prettify())
            
            #article_title = article_soup.find("a", {"href":article_url}).text
            #article_author = article_soup.find_all('p', {"class":"skpzhulc sk3x58id skh9fqbk"})
            #texts = article_soup.find_all('p')
            try:
                driver.get(article_url)
            except:
                continue
            try:
                article_title = driver.find_element_by_class_name('m-entry__title').text
            except:
                try:
                    article_title = driver.find_element_by_class_name('c-page-title').text
                except:
                    try:
                        article_title = driver.find_element_by_class_name('p-page-title').text
                    except:
                        continue
            #article_author = driver.find('p', {"class":"skpzhulc sk3x58id skh9fqbk"})
            texts = driver.find_elements_by_css_selector('p')

            for text in texts:
                try:
                    article_contents += text.text
                except:
                    continue            
            nfl_info["Team"].append(team_dict[query])
            nfl_info["Article URL"].append(article_url)
            nfl_info["Article Title"].append(article_title)
            ##nfl_info["Article Author"].append(article_author[0])
            nfl_info["Text"].append(article_contents)          

    return nfl_info

In [100]:
espn_queries = ["las-vegas-raiders", "kansas-city-chiefs", "chicago-bears", "green-bay-packers", "dallas-cowboys", "philadelphia-eagles"]
espn_team_dict = {"las-vegas-raiders": "raiders", "kansas-city-chiefs":"chiefs", "chicago-bears":"bears", "green-bay-packers":"packers", "dallas-cowboys":"cowboys", "philadelphia-eagles":"eagles"}
espn_nfl_info = defaultdict(list)

for query in espn_queries:
    espn_nfl_info = ESPN_NFLscrape(espn_nfl_info, query, espn_team_dict)

In [101]:
df_espn_nfl = pd.DataFrame(espn_nfl_info)
df_espn_nfl = df_espn_nfl.drop_duplicates("Text")
df_espn_nfl.to_csv("ESPN.csv")

In [86]:
SB_queries = ["https://www.silverandblackpride.com/", "https://www.acmepackingcompany.com/", "https://www.bleedinggreennation.com/", "https://www.arrowheadpride.com/", "https://www.windycitygridiron.com/", "https://www.bloggingtheboys.com/"]
SB_team_dict = {"https://www.silverandblackpride.com/": "raiders", "https://www.arrowheadpride.com/":"chiefs", "https://www.windycitygridiron.com/":"bears", "https://www.acmepackingcompany.com/":"packers", "https://www.bloggingtheboys.com/":"cowboys", "https://www.bleedinggreennation.com/":"eagles"}
sb_nfl_info = defaultdict(list)

for query in SB_queries:
    sb_nfl_info = SBNation_NFLscrape(sb_nfl_info, query, SB_team_dict, driver)

https://www.silverandblackpride.com/2020/11/16/21569817/week-10-grades#comments
https://www.silverandblackpride.com/2020/11/15/21566413/raiders-37-broncos-12-week-10
https://www.silverandblackpride.com/2020/11/1/21544548/nfl-november-raiders-news-tracker
https://www.silverandblackpride.com/raiders-news/2020/11/13/21563501/raiders-fantasy-football-start-or-sit-week-10-edition#comments
https://www.silverandblackpride.com/2020/11/16/21570571/raiders-coach-jon-gruden-chiefs-after-bye-coincidence#comments
https://www.silverandblackpride.com/2020/11/16/21570246/raiders-chiefs-arrowhead-stadium-andy-reid-bus-celebration
https://www.silverandblackpride.com/2020/11/13/21564465/raiders-linebacker-cory-littleton-replaced-nicholas-morrow-denver
https://www.silverandblackpride.com/2020/11/13/21561048/friday-open-thread
https://www.silverandblackpride.com/silver-amp-black-pride-pod
https://www.silverandblackpride.com/2020/11/16/21570231/derek-carrs-legs
https://www.silverandblackpride.com/2020/11/12

In [87]:
df_sb_nfl = pd.DataFrame(sb_nfl_info)
df_sb_nfl = df_sb_nfl.drop_duplicates("Text")
df_sb_nfl.to_csv("SB_Nation.csv")

In [102]:
si_nfl_info = defaultdict(list)
queries = ["packers","bears","eagles","cowboys","raiders","chiefs"]
for query in queries:
    page_url = "https://www.si.com/nfl/" + query
    si_nfl_info = SI_NFLscrape(si_nfl_info, query, driver)

In [103]:
df_si_nfl = pd.DataFrame(si_nfl_info)
df_si_nfl = df_si_nfl.drop_duplicates("Text")
df_si_nfl.to_csv("SI.csv")

In [109]:
df_nfl = pd.concat([df_espn_nfl, df_sb_nfl, df_si_nfl])
df_nfl = df_nfl.drop_duplicates("Text")
df_nfl.to_csv("sports_journalism.csv")

In [123]:
#def FiveThirtyEight_NFLscrape(nfl_info, query, team_dict):
#    page_url = "https://fivethirtyeight.com/?s=" + query
#
#    response = requests.get(page_url)
#    soup = bsoup(response.text, 'lxml')
#    fte_url = re.compile("https://fivethirtyeight.com/features/\w+")
#
#    for posting in set(soup.find_all('a')):
#        article_contents = ""
#        article_url = posting.get('href')
#        if fte_url.match(str(article_url)):
#            print(article_url)
#            try:
#                driver.get(article_url)
#            except:
#                continue
#            try:
#                article_title = driver.find_element_by_class_name('m-entry__title').text
#            except:
#                try:
#                    article_title = driver.find_element_by_class_name('c-page-title').text
#                except:
#                    try:
#                        article_title = driver.find_element_by_class_name('p-page-title').text
#                    except:
#                        continue
#            #article_author = driver.find('p', {"class":"skpzhulc sk3x58id skh9fqbk"})
#            texts = driver.find_elements_by_css_selector('p')
#
#            for text in texts:
#                try:
#                    article_contents += text.text
#                except:
#                    continue            
#            nfl_info["Team"].append(team_dict[query])
#            nfl_info["Article URL"].append(article_url)
#            nfl_info["Article Title"].append(article_title)
#            ##nfl_info["Article Author"].append(article_author[0])
#            nfl_info["Text"].append(article_contents)             
#    
#    return nfl_info 